In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [15]:
with open('input.txt', 'r') as f:
    text = f.read()

In [16]:
print(len(text))

1115394


In [17]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [18]:
# All the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('total chars: ', vocab_size)
print(''.join(chars))

total chars:  65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [19]:
# Encoding and Decoding just using indices
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda x: ''.join([itos[i] for i in x])

In [20]:
# Test encoding and decoding
string = 'hello world'

print(encode(string))
print(decode(encode(string)))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [21]:
# Encode the whole dataset and store it in a tensor
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [22]:
# Split into train and validation sets
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [23]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context}, target is {target}")

When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [45]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences to train on in parallel
block_size = 8 # What is the maximum context length for predictions

def get_batch(split):
    # genereate a small batch of data of inputs x and targeets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [59]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        # Compute the loss using the cross entropy
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus on the last timestep
            logits = logits[:, -1, :]
            # apply softmax to get probs
            probs = F.softmax(logits, dim=1)
            # sample from probs
            idx_next = torch.multinomial(probs, num_samples=1)
            # append the sampled character to running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.4891, grad_fn=<NllLossBackward0>)

g;P;RrlmNHH u SIEWjH;zVw!.OnhNf,WbeuYZ
ezvxAc.TxpvdygBnYLDmVTUkvz'Ka'R r;cXOb3h-ZFqdJleecEgqVN?&HCiw


In [60]:
# create a PyTorch optimizer
optimzier = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
batch_size = 32
for steps in range(30000):
    
    # sample a bacth of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimzier.zero_grad(set_to_none=True)
    loss.backward()
    optimzier.step()
    
print(loss.item())

2.489769697189331


In [69]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


LAUS:
LLUSenkno list LIZAng me croin rts sids y nomyo anereen RLI yoy tou?
UCHe OMENGUCHe amer ithes
